# STEP 0: Use GPU

In [1]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()

True

In [2]:
# Import your package and check the version

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

from models.common import DetectMultiBackend

# You must import the below 5 packages 
import os
import cv2
import time
from tqdm import tqdm
from utils.torch_utils import select_device, time_sync
from utils.general import check_file, check_img_size, check_imshow, non_max_suppression, scale_coords, xyxy2xywh
from utils.datasets import LoadImages

1.9.0+cu102 True


In [7]:
# Initialize the model
device = ''
device = select_device(device)
model = DetectMultiBackend("./runs/train/fish_yolov5x_pre_640_G075/weights/best.pt", device=device, dnn=False)
stride, names, pt, jit, onnx = model.stride, model.names, model.pt, model.jit, model.onnx
if pt and device.type != 'cpu':
    model(torch.zeros(1, 3, 640, 480).to(device).type_as(next(model.model.parameters())))  # warmup

YOLOv5 🚀 25f814d torch 1.9.0+cu102 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)

Fusing layers... 
Model Summary: 444 layers, 86220517 parameters, 0 gradients


In [8]:
from torch import nn
from torchvision import transforms

classification_model_path = "../checkpoint/swin_crop/checkpoint.pth.tar"
model_classification = torch.load(classification_model_path).eval().to(device)
smax = nn.Softmax(dim=1)
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize,
    ]
)

def classification(img):
    with torch.no_grad():
        img = transform(img)
        avg_B = torch.mean(img[0])
        avg_G = torch.mean(img[1])
        avg_R = torch.mean(img[2])
        if (avg_G - 0.05 > avg_B and avg_G - 0.05 > avg_R):
            img[1] = img[1] * 0.75
        data = img.unsqueeze(0).to(device)
        output = model_classification(data)
        output = smax(output).detach().cpu().numpy()
    return output

In [ ]:
import csv
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
# Use the results from your model to generate the output json file
data_path_stg1 = "../../../DATA/fish/test_stg1/"
data_path_stg2 = "../../../DATA/fish/test_stg2/"
test_path = ["../../../DATA/fish/train/DOL/", "../../../DATA/fish/train/YFT/", "../../../DATA/fish/train/NoF/"]
dataset_stg1 = LoadImages(data_path_stg1, img_size=(640,480), stride=stride, auto=pt and not jit)
dataset_stg2 = LoadImages(data_path_stg2, img_size=(640,480), stride=stride, auto=pt and not jit)

conf_thres=0.25
iou_thres=0.45
max_det=100
agnostic_nms=False
classes=None
# Blue color in BGR
color = (255, 0, 0)
thickness = 2


copy_path_stg1 = "../../../DATA/fish/ToSwin/test_stg1/"
copy_path_stg2 = "../../../DATA/fish/ToSwin/test_stg2/"

def copyNoFishImg(image_path, stg):
    src = ""
    output = ""
    if stg == 1:
        src = data_path_stg1 + image_path
        output = copy_path_stg1 + image_path
    else:
        src = data_path_stg2 + image_path
        output = copy_path_stg2 + image_path
    img = cv2.imread(src)
    cv2.imwrite(output, img)


# for each test image
def inference_to_csv(dataset, writer, stg):
    for img_path, img, im0s, vid_cap, s in tqdm(dataset):
    # for img_name in data_listdir:
        # the image_name is as same as the image_id
        img_name = os.path.basename(img_path)
        if (stg == 2):
            image_id = int(img_name[6:-4])
        elif (stg == 1):
            image_id = int(img_name[4:-4])
        # add each detection box infomation into list
        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp16/32
        img /= 255  # 0 - 255 to 0.0 - 1.0
        avg_B = torch.mean(img[0])
        avg_G = torch.mean(img[1])
        avg_R = torch.mean(img[2])
        if (avg_G - 0.05 > avg_B and avg_G - 0.05 > avg_R):
            img[1] = img[1] * 0.75
        if len(img.shape) == 3:
            img = img[None]  # expand for batch dim
        pred = model(img)
        pred_max = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        det = pred_max[0]
        pred = pred_max
        gn = torch.tensor(im0s.shape)[[1, 0, 1, 0]]
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
        
        pre_value = [0,0,0,0,0,0,0,0]
        # for det_box in all_det_boxes_in_this_image:
#         for *xyxy, conf, cls in reversed(det):
#             pre_value[int(cls.cpu().numpy())] = max(float(conf.cpu().numpy()), pre_value[int(cls.cpu().numpy())])
#             pre_value[4] = 1. - pre_value[int(cls.cpu().numpy())]
        for *xyxy, conf, cls in reversed(det):
            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
            w = int(xywh[2] * im0s.shape[1])
            h = int(xywh[3]* im0s.shape[0])
            x = int((xywh[0] * im0s.shape[1]) - (w/2))
            y = int((xywh[1] * im0s.shape[0]) - (h/2))
            crop_img = im0s[y:y+h, x:x+w]
            classification_pre = classification(crop_img)[0]
            pre_value[0] = max(float(classification_pre[0]), pre_value[0])
            pre_value[1] = max(float(classification_pre[1]), pre_value[1])
            pre_value[2] = max(float(classification_pre[2]), pre_value[2])
            pre_value[3] = max(float(classification_pre[3]), pre_value[3])
            pre_value[5] = max(float(classification_pre[4]), pre_value[5])
            pre_value[6] = max(float(classification_pre[5]), pre_value[6])
            pre_value[7] = max(float(classification_pre[6]), pre_value[7])
        for i in range(len(pre_value)):
            if i == 1 or i == 7:
                pre_value[i]+=0.05
            else:
                pre_value[i]+=0.01
        if (len(det) == 0):
#             copyNoFishImg(img_name, stg)
#             continue
            pre_value[4] = float(1)
        if (stg == 2):
            img_name = "test_stg2/" + img_name
        row = [img_name] + pre_value
        writer.writerow(row)
   

with open("answer.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image", "ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"])
    inference_to_csv(dataset_stg1, writer, 1)
    inference_to_csv(dataset_stg2, writer, 2)
#     for p in test_path:
#         dataset_p = LoadImages(p, img_size=(640,480), stride=stride, auto=pt and not jit)
#         inference_to_csv(dataset_p, writer, 1)

 52%|█████▏    | 6323/12153 [07:16<07:16, 13.35it/s]